In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

In [2]:
df = pd.read_csv('catalog4/cat4.csv')
df=df.drop(['Unnamed: 0', 'spectrometric_redshift','pred'],axis=1)

In [3]:
df.head()

,galex_objid,sdss_objid,u,g,r,i,z,extinction_u,extinction_g,extinction_r,...,u-g,u-r,u-i,u-z,g-r,g-i,g-z,r-i,r-z,i-z
0,6.374821e+18,1.237667e+18,20.848124,20.591719,20.275167,20.003620,20.300451,0.075471,0.058807,0.040682,...,0.256405,0.572956,0.844503,0.547672,0.316551,0.588099,0.291267,0.271547,-0.025284,-0.296831
1,6.374821e+18,1.237667e+18,20.177017,20.131323,20.197830,20.047293,19.919939,0.082039,0.063924,0.044223,...,0.045694,-0.020813,0.129725,0.257078,-0.066507,0.084030,0.211384,0.150537,0.277891,0.127354
2,6.374821e+18,1.237667e+18,21.058020,20.534431,20.291599,20.443823,20.201283,0.081629,0.063604,0.044001,...,0.523588,0.766420,0.614197,0.856737,0.242832,0.090609,0.333149,-0.152224,0.090317,0.242540
3,6.374821e+18,1.237667e+18,20.866688,20.533670,20.257778,19.890251,19.691072,0.083731,0.065242,0.045134,...,0.333017,0.608910,0.976437,1.175615,0.275892,0.643419,0.842598,0.367527,0.566706,0.199179
4,6.374821e+18,1.237667e+18,20.024755,20.000088,20.171259,20.368889,20.472017,0.083741,0.065250,0.045140,...,0.024668,-0.146503,-0.344133,-0.447262,-0.171171,-0.368801,-0.471930,-0.197630,-0.300758,-0.103128


In [4]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [5]:
train_df, test_df = train_test_split(df, test_size=20)

In [6]:
#train_df.head()
test_df.head()

,galex_objid,sdss_objid,u,g,r,i,z,extinction_u,extinction_g,extinction_r,...,u-g,u-r,u-i,u-z,g-r,g-i,g-z,r-i,r-z,i-z
18443,6.376193e+18,1.237680e+18,19.618458,19.633221,19.662514,19.423269,19.478884,0.251495,0.195963,0.135567,...,-0.014763,-0.044056,0.195189,0.139574,-0.029293,0.209951,0.154337,0.239244,0.183630,-0.055614
11343,2.916681e+18,1.237679e+18,22.121004,21.406338,21.389154,21.382645,21.454012,0.132909,0.103562,0.071644,...,0.714666,0.731850,0.738359,0.666992,0.017183,0.023693,-0.047674,0.006510,-0.064857,-0.071367
15026,6.380943e+18,1.237680e+18,26.697811,22.791977,21.977354,20.352734,19.264055,0.116692,0.090925,0.062902,...,3.905834,4.720457,6.345078,7.433756,0.814623,2.439243,3.527922,1.624620,2.713299,1.088678
33134,3.864548e+18,1.237664e+18,22.235373,21.379004,21.242704,21.077730,20.763510,0.130409,0.101614,0.070296,...,0.856369,0.992668,1.157642,1.471863,0.136299,0.301273,0.615494,0.164974,0.479195,0.314220
7985,6.380979e+18,1.237679e+18,22.916334,21.912992,21.787579,21.736113,21.419231,0.325673,0.253762,0.175552,...,1.003342,1.128756,1.180222,1.497103,0.125414,0.176880,0.493761,0.051466,0.368347,0.316881


In [7]:

def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification


def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):        # excluding the last column which is the label
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values >  split_value]
    
    return data_below, data_above



In [8]:
import pandas as pd
df=pd.read_csv('project/dataset/catalog1/cat1.csv')
#df.head()
#df.columns
df=df.drop(['Unnamed: 0', 'spectrometric_redshift','pred'],axis=1)
#df.head()
#df.head()
target=df['class']
target
df=df.drop(['class'],axis=1)


# def check_purity(data):
    
#     label_column = data[:, -1]
#     unique_classes = np.unique(label_column)

#     if len(unique_classes) == 1:
#         return True
#     else:
#         return False
    
# check_purity(df)

In [9]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [10]:

def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df           
    
#     data=df
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # instantiate sub-tree
        feature_name = COLUMN_HEADERS[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [11]:

def calculate_accuracy(df, tree):

    df["pred_class"] = df.drop(["class"],axis=1).apply(classify_example, axis=1, args=(tree,))
    
    df["pred_class_correct"] = df["pred_class"] == df["class"]
    print(df.head())
    accuracy = df["pred_class_correct"].mean()
    
    return accuracy

In [12]:
df = pd.read_csv("catalog4/cat4.csv")
df=df.sample(n=10000)
df_old=df
df=df.drop(['Unnamed: 0', 'spectrometric_redshift','pred'],axis=1)
#df.head()
#df.head()
target=df['class']
target

train_df, test_df = train_test_split(df, test_size=0.2)

#train_df=df.drop(['class'],axis=1)

Y_train=train_df["class"]
train_df=train_df.drop(["class"],axis=1)
train_df["class"]=Y_train

#df.shape
#train_df.shape
#train_df.head()

In [ ]:
tree = decision_tree_algorithm(train_df)
tree
#accuracy = calculate_accuracy(test_df, tree)

In [ ]:
accuracy = calculate_accuracy(test_df, tree)

In [ ]:
accuracy